## Set up Web3

### Provider

Here we are using infura in the example. You may change it to any other provider.

In [ ]:
import os
import json
import time
import pandas as pd
import local_config
from web3.auto.infura.rinkeby import w3

os.environ['WEB3_INFURA_PROJECT_ID'] = local_config.INFURA_PROJECT_ID

w3.isConnected()

### PredictionRecorder contract to interact with

In [ ]:
RECORDER_ADDRESS = local_config.PREDICTION_RECORDER_CONTRACT

### Your Account

This example loads private keys in a simplistic manner. **We suggest using a test account with no real asset.**

In [ ]:
ACCOUNT_ADDRESS = local_config.PUBLIC_ADDRESS_DUMMY
ACCOUNT_KEY = local_config.SIGNER_PRIVATE_KEY_DUMMY

## Load the contract and connect it to Python API

In [ ]:
import requests

abi = requests.get(f"https://api-rinkeby.etherscan.io/api?module=contract&action=getabi&format=raw&address={RECORDER_ADDRESS}").text
abi

In [ ]:
from credict.api import PredictionHandler

contract_dict = dict(abi=abi, address=RECORDER_ADDRESS)

target_json_path = PredictionHandler.json_path_static(ACCOUNT_ADDRESS, RECORDER_ADDRESS)

if os.path.isfile(target_json_path):
    print("Loading handler state from existing json.")
    handler = PredictionHandler.from_json(target_json_path)
else:
    print("Did not find existing json. Creating new handler.")
    handler = PredictionHandler(ACCOUNT_ADDRESS, contract_dict)

In [ ]:
if not hasattr(handler, "rsa"):
    handler.setup_rsa()
handler.connect(w3)
handler.rsa.describe()

In [ ]:
def transact(call):
    txn = call.buildTransaction({
        "value": 0,
        "gas": int(1e+6),
        "nonce": w3.eth.get_transaction_count(ACCOUNT_ADDRESS),
        'gasPrice': int(w3.eth.gas_price * 1.1),
    })
    signed_txn = w3.eth.account.sign_transaction(txn, private_key=ACCOUNT_KEY)
    return w3.eth.send_raw_transaction(signed_txn.rawTransaction)

## Send predictions

In [ ]:
predict_receipt = handler.make_prediction(
    # transaction callback
    transact,
    # prediction target time example: 24 hours from now
    int(time.time()) + 86400,
    # un-encrypted prediction value; credict-py handles the encryption
    2000,
    # other arguments
    author="John Doe",
    comment="This prediction was based on <analysis> and <assumption>",
)
predict_receipt

In [ ]:
pd.DataFrame(handler.view_prediction())

## Decrypt predictions

In [ ]:
handler.records_to_decrypt

In [ ]:
decrypt_receipt = handler.decrypt_prediction(transact)
decrypt_receipt

## Check forecaster stats

In [ ]:
handler.contract.functions.analyzePredictionRecord(ACCOUNT_ADDRESS).call()